# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [231]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
r = requests.get(url)

data_json = r.json()

json_dataframe = pd.DataFrame(data_json['items'])

pvl_rj = json_dataframe[(json_dataframe['uf'] == 'RJ') & (json_dataframe['tipo_interessado'] == 'Município')]

pvl_rj.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
23,34099,Município,Resende,3304201,RJ,PVL02.003090/2019-99,Deferido (PVL-IF),None,None,Operação contratual interna,Aporte de capital,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4500000.0,0,1,27/09/2019


In [233]:
# 2) Seu código aqui
contagem_status = pvl_rj['status'].value_counts()
contagem_status.head(3)

status
Deferido             18
Arquivado            12
Deferido (PVL-IF)     7
Name: count, dtype: int64

In [235]:
# 3) Seu código aqui

data_anos = pvl_rj['data_status'].str.split('/').str[-1]
data_anos

23      2019
76      2012
94      2014
397     2014
407     2012
454     2013
463     2008
547     2007
633     2014
865     2008
867     2014
1043    2008
1282    2013
1306    2023
1380    2020
1402    2021
1469    2023
1495    2013
1528    2008
1742    2013
1865    2013
1930    2008
2159    2008
2165    2011
2340    2009
2524    2011
2657    2020
2693    2019
2801    2007
3140    2011
3238    2008
3333    2015
3357    2022
3404    2019
3434    2008
3456    2008
3467    2023
3475    2022
3495    2019
3555    2007
3762    2008
3823    2019
3963    2023
3998    2014
4020    2013
4032    2010
4260    2014
4338    2015
4616    2023
4931    2023
Name: data_status, dtype: object

In [237]:
data_anos.value_counts()

data_status
2008    10
2014     6
2013     6
2023     6
2019     5
2007     3
2011     3
2012     2
2020     2
2015     2
2022     2
2021     1
2009     1
2010     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [165]:
#1) Seu código aqui

def consulta_estado_municipio(tipo_interessado, interessado):
    
    df_filtrado = json_dataframe[
        (json_dataframe['tipo_interessado'].str.upper() == tipo_interessado.upper()) &
        (json_dataframe['interessado'].str.upper() == interessado.upper())
    ]
    
    if df_filtrado.empty:
        return f'{interessado} não foi encontrado no nosso banco de dados'
    else:
        return df_filtrado

consulta_estado_municipio('Município','Pranchita')

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,44971,Município,Pranchita,4120358,PR,PVL02.001181/2021-12,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Agência de Fomento do Paraná S/A,Real,2000000.0,0,1,26/11/2021
1119,16418,Município,Pranchita,4120358,PR,None,Deferido,19406.000303/2008-53,2008-04-22T03:00:00Z,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Agência de Fomento do Paraná S/A,Real,70000.0,1,0,29/04/2008
3568,49189,Município,Pranchita,4120358,PR,PVL02.004961/2021-14,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Agência de Fomento do Paraná S/A,Real,1200000.0,0,1,26/11/2021


In [195]:
# 2) Seu código aqui

((json_dataframe['uf'].str.upper() == 'MG') & (json_dataframe['status'].str.upper() == ('Arquivado por decurso de prazo').upper())).sum()

87

In [219]:
# 3) Seu código aqui

deferido_filtrado = json_dataframe[((json_dataframe['tipo_interessado'].str.upper() == ('Município').upper()) & (json_dataframe['uf'].str.upper() == 'BA') & (json_dataframe['status'].str.upper() == ('Deferido').upper()))]
deferido_filtrado_contagem = deferido_filtrado['interessado'].value_counts()
deferido_filtrado_contagem.head(1)

interessado
Camaçari    3
Name: count, dtype: int64

In [229]:
# 4) Seu código aqui
BA_filtrado = consulta_estado_municipio('Estado','Bahia')
BA_filtrado.to_csv('Estado_Bahia_filtrado.csv')
BA_filtrado

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
332,44369,Estado,Bahia,29,BA,PVL02.000482/2021-11,Deferido,17944.101324/2021-40,2021-05-03T19:36:30Z,Concessão de garantia,Concessão de garantia a empresa estatal não de...,Outras Organizações,Pessoa Jurídica,Real,4.990000e+08,0,0,17/05/2021
1294,10198,Estado,Bahia,29,BA,None,Deferido,17944.001142/2010-17,2010-08-03T03:00:00Z,Operação contratual interna,PMAE,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,9.905400e+06,1,0,16/08/2010
1576,18559,Estado,Bahia,29,BA,None,Deferido,19407.000051/2002-67,2002-06-11T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.983597e+07,1,0,08/07/2002
1577,18567,Estado,Bahia,29,BA,None,Deferido,19407.000053/2002-56,2002-06-17T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.039426e+06,1,0,03/07/2002
1578,18571,Estado,Bahia,29,BA,None,Deferido,19407.000054/2002-09,2002-06-17T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.797522e+05,1,0,08/07/2002
2117,11283,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001402/2009-11,2010-03-02T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.832790e+08,1,0,09/03/2010
2365,10859,Estado,Bahia,29,BA,None,Deferido,17944.001302/2013-71,2013-10-11T03:00:00Z,Operação contratual interna,PAC 2 - Mobilidade Grandes Cidades,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4.000000e+08,1,0,18/10/2013
2410,12991,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001875/2008-37,2010-05-10T03:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.000000e+07,1,0,24/06/2010
2672,18519,Estado,Bahia,29,BA,None,Deferido,19407.000041/2002-21,2002-06-11T03:00:00Z,Operação contratual interna,Multissetorial,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.313312e+06,1,0,05/07/2002
2735,18547,Estado,Bahia,29,BA,None,Deferido,19407.000048/2002-43,2002-06-11T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.400843e+06,1,0,03/07/2002
